- [ ] sheet 2 intl departure needs to be taken into account 
- [ ] eng arrival and base departure needs to be finalised
- [ ] move entire code into seperate repositiory 
- [ ] understand stremlit 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import pickle
import re
from vega_datasets import data

temp_list = pickle.load(open("gsheet_pickle.p", "rb"))
# importing from pickle 
alpha = temp_list[0]
# seperate master datframe being created for all details after all modification
master_df = pd.DataFrame(columns =['Dept Arp', 'engg_arr', 'engg_dep', 'deptype', 'Arvl Arp', 'Flt Desg',
       'Block Time', 'aocs_dep', 'aocs_arr', 'Text Comment'])




In [2]:
alpha.head()


,Dept Arp,engg_arr,deptype,Arvl Arp,Flt Desg,Text Comment,Block Time,variable,value
0,IXC,False,False,GOI,6E 0389,A10,2:45,D1,2021-02-01 14:25:00
1,GOI,False,False,IXC,6E 0391,A10,2:40,D1,2021-02-01 17:40:00
2,BLR,False,False,RPR,6E 0378,A100,2:00,D1,2021-02-01 08:30:00
3,RPR,False,False,BLR,6E 0382,A100,1:35,D1,2021-02-01 11:10:00
4,BLR,False,False,VGA,6E 0507,A100,1:20,D1,2021-02-01 13:20:00


In [3]:
# pseudo code 
# sort as per column "D2 "
# last row copy and add a new row for base arrival case 
# first row  make engg equal to D2 
# all middle rows make engg arrival  previous row A2 
# add a day column at the last reprenting the day 

In [4]:

def make_last_row(onedf ,arr_D):
    temp_false_list = []
    for india in range(onedf.shape[1]):
        temp_false_list.append(False)
    temp_false_list
    onedf.loc[len(onedf)] = temp_false_list
    onedf.loc[(len(onedf)-1), "Dept Arp" ] =onedf.loc[(len(onedf)-2), "Arvl Arp" ]
    onedf.loc[(len(onedf)-1), "engg_arr" ] =onedf.loc[(len(onedf)-2), arr_D]
    onedf.loc[(len(onedf)-1), "Text Comment" ] =onedf.loc[(len(onedf)-2), "Text Comment" ]
    return onedf
# right_df = make_last_row(right_df,arrDay)
# right_df





In [5]:
def AddBaseArrDep(onedf ,dep_day,arr_day):
    '''
    Modify first and last row for base Departure and base arrival model 
    '''
    for xray in range(0,len(onedf)):
        if xray  == 0:
            onedf.loc[xray,"deptype"] = "baseDep"
            onedf.loc[xray,"engg_arr"] = "baseAcft"
            onedf.loc[xray,"engg_dep"] = onedf.loc[xray,dep_day]
        elif xray == len(onedf)-1:
            
            onedf.loc[xray,"deptype"] = "baseArr"
           
        else:
            onedf.loc[xray,"engg_arr"] = onedf.loc[(xray-1),arr_day]
            onedf.loc[xray,"engg_dep"] = onedf.loc[(xray),dep_day]
            onedf.loc[xray,"deptype"] = "Transit"
    return onedf



# onedf.columns






In [6]:
# depDay = "D4"
# arrDay = "A4"
# textcomment is constant and is under manual 
# x_df =alpha.loc[(alpha["Text Comment"]=="A10") & ((alpha["variable"]==depDay) | (alpha["variable"]==arrDay) )]
# removing false entries and sorting as false entries cannot be sorted 

def df_flat_function(test_df , dep_D,arr_D):
    test_df = test_df.sort_values(by="value")
    test_df =test_df.pivot(index = ['Dept Arp', 'engg_arr', 'deptype', 'Arvl Arp', 'Flt Desg','Text Comment', 'Block Time'] ,columns = "variable" ,values = 'value')
    test_df = test_df.sort_values(by=dep_D)
    flattened = pd.DataFrame(test_df.to_records())
    flattened["engg_dep"]=False
    flattened =  flattened[['Dept Arp', 'engg_arr','engg_dep' ,'deptype', 'Arvl Arp', 'Flt Desg','Block Time', dep_D, arr_D,'Text Comment']]
    
    
    return flattened


# right_df

In [7]:
# looping over for one month and keeping the comment constant manually 
# for india in range(1,32):
#     depDay = "D"+str(india)
#     arrDay = "A" +str(india)
    
#     x_df =alpha.loc[(alpha["Text Comment"]=="A10") & ((alpha["variable"]==depDay) | (alpha["variable"]==arrDay) )]
#     if ((depDay in x_df["variable"].unique()) and (arrDay in x_df["variable"].unique())):
#         right_df = df_flat_function(x_df ,depDay,arrDay)
#         make_last_row(right_df,arrDay)
#         AddBaseArrDep(right_df,depDay,arrDay)
#         right_df = right_df.rename(columns={depDay:'aocs_dep',arrDay:"aocs_arr"})
#         master_df= master_df.append(right_df,ignore_index=True)



In [8]:

master_df.head(40)

,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,Block Time,aocs_dep,aocs_arr,Text Comment


In [9]:
%%time

for iran in list(alpha["Text Comment"].unique()):
    for india in range(1,32):
        depDay = "D"+str(india)
        arrDay = "A" +str(india)
    
        x_df =alpha.loc[(alpha["Text Comment"]== iran) & ((alpha["variable"]==depDay) | (alpha["variable"]==arrDay) )]
        if ((depDay in x_df["variable"].unique()) and (arrDay in x_df["variable"].unique())):
                right_df = df_flat_function(x_df ,depDay,arrDay)
                make_last_row(right_df,arrDay)
                AddBaseArrDep(right_df,depDay,arrDay)
                right_df = right_df.rename(columns={depDay:'aocs_dep',arrDay:"aocs_arr"})
                master_df= master_df.append(right_df,ignore_index=True)



Wall time: 9min 9s


In [10]:
master_df.head()

,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,Block Time,aocs_dep,aocs_arr,Text Comment
0,IXC,baseAcft,2021-02-01 14:25:00,baseDep,GOI,6E 0389,2:45,2021-02-01 14:25:00,2021-02-01 17:10:00,A10
1,GOI,2021-02-01 17:10:00,2021-02-01 17:40:00,Transit,IXC,6E 0391,2:40,2021-02-01 17:40:00,2021-02-01 20:20:00,A10
2,IXC,2021-02-01 20:20:00,False,baseArr,False,False,False,False,False,A10
3,IXC,baseAcft,2021-02-02 07:35:00,baseDep,PNQ,6E 6514,2:25,2021-02-02 07:35:00,2021-02-02 10:00:00,A10
4,PNQ,2021-02-02 10:00:00,2021-02-02 10:30:00,Transit,IXC,6E 6515,2:30,2021-02-02 10:30:00,2021-02-02 13:00:00,A10


In [11]:
len(list(alpha["Text Comment"].unique()))
    


261

In [12]:
newdf = master_df
newdf["depDay"]=False
for xray in range(0,len(newdf)):
    if newdf.loc[xray,"engg_dep"] != False:
        newdf.loc[xray,"depDay"] = "D" +str(newdf.loc[xray,"engg_dep"].day)
    elif newdf.loc[xray,"engg_dep"] == False:
        newdf.loc[xray,"depDay"] = "D" +str(newdf.loc[xray,"engg_arr"].day)

In [13]:
pickle.dump(newdf, open("minortest_pickle.p", "wb"))

gamma = pickle.load(open("minortest_pickle.p", "rb"))

gamma.equals(newdf)

True

In [14]:
# Manual Testing 

test_alpha = gamma.sample(n=5)
test_alpha

,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,Block Time,aocs_dep,aocs_arr,Text Comment,depDay
12266,HYD,2021-02-06 15:25:00,2021-02-06 16:50:00,Transit,TRZ,6E 7318,1:55,2021-02-06 16:50:00,2021-02-06 18:45:00,A320_110,D6
22178,HYD,2021-02-15 19:20:00,2021-02-15 20:20:00,Transit,BLR,6E 6916,1:10,2021-02-15 20:20:00,2021-02-15 21:30:00,A38,D15
13498,BLR,2021-02-20 23:35:00,False,baseArr,False,False,False,False,False,A320_201,D20
27516,HYD,2021-02-20 23:05:00,False,baseArr,False,False,False,False,False,A70,D20
32861,CNN,2021-02-15 15:15:00,2021-02-15 16:05:00,Transit,BLR,6E 7973,1:15,2021-02-15 16:05:00,2021-02-15 17:20:00,ATR003,D15


In [2]:
source = data.stocks()
    
alt.Chart(source).mark_line().encode(
        x='date',
        y='price',
        color='symbol',
        strokeDash='symbol',
    )

NameError: name 'data' is not defined